<a href="https://colab.research.google.com/github/Doclikam/Causal-inference-for-Treatment-Effects-in-Head-Neck-Radiotherapy./blob/main/RADCURE_causal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RADCURE - CHEMOTHERAPY/RADIOTHERAPY HEAD AND NECK CANCER RESPONSE**

In [ ]:
#main libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
from datetime import datetime

# Display all columns
pd.set_option('display.max_columns', None)

In [ ]:
radcure_data = pd.read_excel("/content/RADCURE_Clinical_v04_20241219.xlsx")#load the excel file
radcure_data.head(2)#visulize the first 2 patients

,patient_id,Age,Sex,ECOG PS,Smoking PY,Smoking Status,Ds Site,Subsite,T,N,M,Stage,Path,HPV,Tx Modality,Chemo,RT Start,Dose,Fx,Last FU,Status,Length FU,Date of Death,Cause of Death,Local,Date Local,Regional,Date Regional,Distant,Date Distant,2nd Ca,Date 2nd Ca,RADCURE-challenge,ContrastEnhanced
0,RADCURE-0005,62.6,Female,ECOG 0,50,Ex-smoker,Oropharynx,post wall,T4b,N2c,M0,IVB,Squamous Cell Carcinoma,"Yes, Negative",RT alone,none,2002-02-20,60.0,25,2003-05-12,Dead,1.317808,2003-05-12,Other Cause,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,0
1,RADCURE-0006,87.3,Male,ECOG 2,25,Ex-smoker,Larynx,Glottis,T1b,N0,M0,I,Squamous Cell Carcinoma,NaN,RT alone,none,2006-01-17,51.0,20,2007-06-28,Dead,1.520548,2007-06-28,Other Cause,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0,1


In [ ]:
#standardize column names
radcure_data.columns = radcure_data.columns.str.strip().str.lower()

In [ ]:
#Columns
radcure_data.columns

Index(['patient_id', 'age', 'sex', 'ecog ps', 'smoking py', 'smoking status',
       'ds site', 'subsite', 't', 'n', 'm', 'stage', 'path', 'hpv',
       'tx modality', 'chemo', 'rt start', 'dose', 'fx', 'last fu', 'status',
       'length fu', 'date of death', 'cause of death', 'local', 'date local',
       'regional', 'date regional', 'distant', 'date distant', '2nd ca',
       'date 2nd ca', 'radcure-challenge', 'contrastenhanced'],
      dtype='object')

In [ ]:
rad_data = radcure_data.copy()

# Define Exposure (Treatment)

We limit to a **binary** clinical decision made at baseline:

- `ChemoRT`: Chemotherapy + Radiotherapy
- `RT-alone`: Radiotherapy alone

In [ ]:
#treatment modalities
rad_data['tx modality'].value_counts()

In [ ]:
#harmonize the treatment modality from 'RT alone', 'ChemoRT', 'RT + EGFRI', 'ChemoRT ', 'Postop RT alone'
rad_data["tx_modality"] = rad_data["tx modality"].str.lower().str.strip()

rad_data['tx_modality_clean'] = rad_data['tx_modality'].replace({
    'rt alone': 'rt_alone',
    'chemort': 'chemort',
    'rt + e gfri': 'rt_egfri',  # ensure mapping correct
    'postop rt alone': 'postop_rt'
})


#Our treatrment (Radiotherapy alone vs Radiotherapy +chemotherapy)
#Primary causal exposure: chemo(addon) vs RT only
rad_data['tx_chemo'] = (rad_data['tx_modality_clean'] == 'chemort').astype(int)

# Identify EGFRI for sensitivity analyses
rad_data['tx_egfri'] = (rad_data['tx_modality_clean'] == 'rt_egfri').astype(int)

# Drop Post-operative RT from primary
df = rad_data[rad_data['tx_modality_clean'] != 'postop_rt']

## Survival Outcomes

- Time zero = **RT Start**
- Event = **death from any cause**
- Censoring = Last follow-up if alive
- All times in **days**

Composite endpoints (local/regional/distant recurrence, PFS) are built without removing patients lacking event dates.

In [ ]:
#rename event cols
event_cols = ['rt start', 'last fu','length fu', 'date of death', 'cause of death', 'local',
             'date local', 'regional', 'date regional', 'distant',
             'date distant', '2nd ca', 'date 2nd ca']

# build mapping only for columns that exist in rad_data
rename_map = {
    c: c.strip().lower().replace(" ", "_").replace("-", "_")
    for c in event_cols if c in rad_data.columns
}

rad_data.rename(columns=rename_map, inplace=True)


In [ ]:
#Only date columns to parse (true dates only)
date_cols = [
    "rt_start", "last_fu", "date_of_death",
    "date_local", "date_regional", "date_distant", "date_2nd_ca"
]

for c in date_cols:
    if c in rad_data.columns:
        # Mixed formats are okay; errors='coerce' -> NaT if unparseable
        rad_data[c] = pd.to_datetime(rad_data[c], errors="coerce")

# Overall survival (OS)
rad_data["os_end"] = rad_data["date_of_death"].fillna(rad_data["last_fu"])
rad_data["time_os_days"] = (rad_data["os_end"] - rad_data["rt_start"]).dt.days
rad_data["event_os"] = rad_data["date_of_death"].notna().astype(int)

# QC flags (do not drop — only flag)
rad_data["bad_time_zero"]  = rad_data["rt_start"].isna().astype(int)
rad_data["bad_followup"]   = rad_data["last_fu"].isna().astype(int)
rad_data["neg_or_zero_os"] = (
    rad_data["time_os_days"].isna() | (rad_data["time_os_days"] <= 0)
).astype(int)
rad_data["death_after_fu"] = (
    rad_data["event_os"].eq(1) & (rad_data["date_of_death"] > rad_data["last_fu"])
).astype(int)

# Event flags from the presence of dates (no text parsing needed)
for site in ["local", "regional", "distant", "2nd_ca"]:
    date_col = f"date_{site}"
    if date_col in rad_data.columns:
        rad_data[f"event_{site}"] = rad_data[date_col].notna().astype(int)
    else:
        rad_data[f"event_{site}"] = 0

# Time-to-event (only for rows with that event)
for site in ["local", "regional", "distant"]:
    date_col = f"date_{site}"
    if date_col in rad_data.columns:
        rad_data[f"time_{site}_days"] = np.where(
            rad_data[f"event_{site}"] == 1,
            (rad_data[date_col] - rad_data["rt_start"]).dt.days,
            np.nan
        )

# Progression-Free Survival (first of any recurrence OR death)
pfs_sources = [c for c in ["date_local","date_regional","date_distant","date_of_death"] if c in rad_data.columns]
if pfs_sources:
    pfs_end = pd.concat([rad_data[c] for c in pfs_sources], axis=1).min(axis=1)
    rad_data["time_pfs_days"] = (pfs_end - rad_data["rt_start"]).dt.days
    rad_data["event_pfs"] = (
        (rad_data[["event_local","event_regional","event_distant"]].sum(axis=1) > 0) |
        (rad_data["event_os"] == 1)
    ).astype(int)
    # Censor PFS at last FU if no event
    rad_data.loc[rad_data["event_pfs"] == 0, "time_pfs_days"] = (
        rad_data["last_fu"] - rad_data["rt_start"]
    ).dt.days

# Follow-up (descriptive)
rad_data["followup_days"] = (rad_data["last_fu"] - rad_data["rt_start"]).dt.days

# Sanity: set non-positive times to NaN (keep flagged via QC vars above)
for col in ["time_os_days","time_pfs_days","time_local_days","time_regional_days","time_distant_days","followup_days"]:
    if col in rad_data.columns:
        rad_data.loc[rad_data[col] <= 0, col] = np.nan

# **Confounder processing**

# Preprocessing Confounders

###Smoking — baseline behavioral risk factor

Both pack-years and status influence:
- treatment choice (Chemo vs RT)
- survival


                 

In [ ]:
rad_data['smoking status'].value_counts()

In [ ]:
#Smoking status
rad_data['smoking_status_clean'] = (
    rad_data['smoking status'].astype(str).str.strip().str.title()
)


#We have "na" values in our dataset that are not accounted for in the missingnes and inconsistencies such as >5, <20
# clean smoking pack-year column
rad_data["smoking_py_clean"] = (
    rad_data["smoking py"]
    .astype(str)
    .str.strip()
    .replace({"na": np.nan, "NA": np.nan, "NaN": np.nan, "": np.nan})
)

# Clean pack years strings
def clean_smoking_py(value):
    if pd.isna(value): return np.nan
    v = str(value).strip().lower()

    if v.startswith("<"):
        try: return float(v[1:]) / 2
        except: return np.nan

    if v.startswith(">"):
        try: return min(float(v[1:]) + 5, 120)
        except: return np.nan

    try:
        return float(v)
    except:
        return np.nan

rad_data['smoking_py_clean'] = rad_data['smoking py'].apply(clean_smoking_py)

# Inconsistency flags
rad_data['smoking_mismatch_flag'] = (
    ((rad_data['smoking_status_clean'] == 'Non-Smoker') &
     (rad_data['smoking_py_clean'] > 0)) |
    ((rad_data['smoking_status_clean'].isin(['Ex-Smoker','Current'])) &
     (rad_data['smoking_py_clean'].isna() | (rad_data['smoking_py_clean'] == 0)))
).astype(int)

# Correct inconsistencies
rad_data.loc[
    (rad_data['smoking_status_clean'] == 'Non-Smoker') &
    (rad_data['smoking_py_clean'] > 0),
    'smoking_py_clean'
] = 0 #if person is a non-smoker and has values other than 0 replace with 0

rad_data.loc[
    (rad_data['smoking_status_clean'].isin(['Ex-Smoker','Current'])) &
    (rad_data['smoking_py_clean'] == 0),
    'smoking_py_clean'
] = np.nan #if person has smoked/currenty smoking and has 0 pack years replace with np.nan

# Missingness indicator
rad_data['smoking_py_missing'] = rad_data['smoking_py_clean'].isna().astype(int)


 ### Disease Site And Subsite

 Mapped into **4 clinically meaningful groups**:
- Oropharynx
- Larynx
- Nasopharynx
- Other_HNC

Subsite included **only** for Oro/Larynx where relevant and complete

In [ ]:
rad_data['ds site'].value_counts()

In [ ]:
#lower case for consistent naming of the tumor sites.
rad_data['ds_site'] = rad_data['ds site'].astype(str).str.lower().str.strip()

# map for consistent naming of the DS site
primary_site_map = {
    'nasal cavity': 'nasal cavity',
    'nasal cavity ': 'nasal cavity',
    'esophagus': 'esophagus',
    'lip & oral cavity': 'lip_oral_cavity'
}
rad_data['ds_site_clean'] = rad_data['ds_site'].replace(primary_site_map)

#for casual modelling we will use four main groups.
def primary_group(site):
    if 'oropharynx' in site:
        return "Oropharynx"
    if 'larynx' in site:
        return "Larynx"
    if 'nasopharynx' in site:
        return "Nasopharynx"
    return "Other_HNC"

#apply this changes
rad_data['primary_site_group'] = rad_data['ds_site_clean'].apply(primary_group)

In [ ]:
#handle the missingness of subsite
rad_data['subsite_clean'] = rad_data['subsite'].astype(str).str.strip().str.title()


# we subsite that are flagged as valid; clinically relevant & low missingness
flagged_subsite_sites = ["Oropharynx", "Larynx"]
rad_data.loc[
    ~rad_data['primary_site_group'].isin(flagged_subsite_sites),
    'subsite_clean'
] = np.nan  # Remove meaningless subsites

# Subsite missingness indicator only within valid groups to identify true missingness rather than structural
rad_data['subsite_missing'] = (
    rad_data['subsite_clean'].isna() &
    rad_data['primary_site_group'].isin(flagged_subsite_sites)
).astype(int)

### Tumor Extent at Diagnosis (TNM and Stage)

TNM missingness is **informative**.
We:
- preserve missing values
- add missingness flags
- distinguish **structural** vs **statistical** missing

In [ ]:
#Flag non-TNM cancers (TNM not applicable)
def identify_non_tnm(site):
    site = str(site).lower()
    if any(x in site for x in ['paraganglioma', 'glomus', 'benign', 'orbit', 'lacrimal']):
        return 1
    return 0

rad_data['is_non_tnm'] = rad_data['ds_site_clean'].apply(identify_non_tnm)

#Flag structural TNM missing
# TNM exists but often not assessed in practice for these sites
rad_data['tnm_structural_missing'] = (
    rad_data['is_non_tnm'] |
    rad_data['ds_site_clean'].str.contains('skin|sarcoma|esophagus', case=False)
).astype(int)

#Remove TNM where not applicable (STRUCTURAL missing)
tnm_cols = ['t', 'n', 'm']
for col in tnm_cols:
    rad_data.loc[rad_data['tnm_structural_missing'] == 1, col] = np.nan

#Missingness flags for TNM (STATISTICAL missing)

for col in tnm_cols:
    rad_data[f'{col}_missing'] = (
        rad_data[col].isna() &
        (rad_data['tnm_structural_missing'] == 0)
    ).astype(int)

###HPV Status

HPV is only clinically relevant in **Oropharynx** tumors.

We recode:
- HPV_Positive
- HPV_Negative
- HPV_Unknown

HPV set to **NaN** outside Oropharynx to avoid spurious adjustment.

In [ ]:
rad_data['hpv'].value_counts()

In [ ]:
#Normalize the HPV entries to"HPV_positive","HPV_negative" or HPV_unknown
def clean_hpv(value):
    if pd.isna(value):
        return "HPV_Unknown"

    v = str(value).strip().lower()

    if "pos" in v:
        return "HPV_Positive"
    if "neg" in v:
        return "HPV_Negative"

    return "HPV_Unknown"

rad_data['hpv_clean'] = rad_data['hpv'].apply(clean_hpv)

# HPV specific/ relevant tumors ONLY in Oropharynx
rad_data['hpv_specific_tumors'] = (
    rad_data['primary_site_group'].eq("Oropharynx")
).astype(int)


# If HPV not relevant → set to NaN (avoid incorrect confounding)
rad_data.loc[
    rad_data['hpv_specific_tumors'] == 0,
    'hpv_clean'
] = np.nan


# Missingness flag: only meaningful for relevant cancers
rad_data['hpv_missing'] = (
    rad_data['hpv_clean'].isna() &
    rad_data['hpv_specific_tumors'].eq(1)
).astype(int)

#flag stage missing
rad_data['stage_missing'] = rad_data['stage'].isna().astype(int)

# ECOG

In [ ]:
# ECOG
rad_data['ecog_ps'] = pd.to_numeric(rad_data['ecog ps'], errors='coerce')
rad_data['ecog_ps_missing'] = rad_data['ecog_ps'].isna().astype(int)

# Age


Age ─► Treatment (Chemo vs RT) ─► Outcome (Survival)
 └────► Outcome (Survival)

In [ ]:
# Standardize age (z-score scaling)
rad_data['age_std'] = (rad_data['age'] - rad_data['age'].mean()) / rad_data['age'].std()

# non-linear features for modeling
rad_data['age_std_sq'] = rad_data['age_std'] ** 2

# Downstream treatment components

This include RT dose, dose per fraction, number of fractions, treatment duration, BED/EQD2

# **Final modelling Columns**

In [ ]:
# List of columns to keep in analytic set
analytic_columns = [
    'patient_id',
    'tx_chemo',
    'time_os_days', 'event_os',
    'age_std', 'sex',
    'ecog_ps', 'ecog_ps_missing',
    'smoking_status_clean', 'smoking_py_clean',
    'smoking_py_missing', 'smoking_mismatch_flag',
    'primary_site_group',
    'subsite_clean', 'subsite_missing',
    't', 'n', 'm', 'stage',
    't_missing', 'n_missing', 'm_missing', 'stage_missing',
    'hpv_clean', 'hpv_missing', 'hpv_specific_tumors',
    'is_non_tnm', 'tnm_structural_missing',
    'followup_days'
]

# Filter to analytic set
rad_data_analysis = rad_data[analytic_columns].copy()
rad_data_analysis.to_csv("rad_data_analysis.csv", index=False)
print("✅ Analytic dataset created:", rad_data_analysis.shape)


✅ Analytic dataset created: (3346, 29)


In [ ]:

confounder_model_columns = [
    'age_std', 'sex',
    'ecog_ps', 'ecog_ps_missing',
    'smoking_status_clean', 'smoking_py_clean', 'smoking_py_missing',
    'primary_site_group',
    't', 'n', 'm',
    't_missing', 'n_missing', 'm_missing', 'stage_missing',
    'hpv_clean', 'hpv_missing', 'hpv_specific_tumors']


rad_data_modelling = rad_data[confounder_model_columns].copy()
rad_data_modelling.to_csv("rad_data_modelling.csv", index=False)
print("confounder dataset created imported to csv:", rad_data_modelling.shape)


confounder dataset created imported to csv: (3346, 18)
